In [60]:
import pandas as pd
import pymysql
import cv2
from matplotlib import pyplot as plt
import sys
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import pandas as pd
from pandas import DataFrame
from pylab import *
import numpy

In [61]:
conn = pymysql.connect(host='localhost',user='root',passwd='####',port = 3306,db='taobao',charset='utf8')
cur=conn.cursor

In [69]:
sql = "select * from taobao.image where isSaved_Picture = 1 and position<>'2' limit 5000000;"

image_list = pd.read_sql(sql, con=conn) 

In [5]:
import os
path = '../任务二/3.获取图片'
files= os.listdir(path) #得到文件夹下的所有文件名称

In [63]:
print(image_list.shape)

(315, 10)


In [49]:
def dHash(m,n):
    tmp = pd.DataFrame()
    v_hash = []
    pic_id = []
    for i in range(m,n):
        path = image_list_j.iat[i,9]
        pic_path = 'D:\\image\\' + path + '.jpg';
        pic_path2 = ''
        pic_id_value = image_list_j.iat[i,8]
        if pic_path in files:
            pic_path2 = '../任务二/3.获取图片/' + pic_path  
        if len(pic_path2) == 0:
            print('E1')
            v_hash.append(None)
            pic_id.append(None)
            continue
        try:
            
            img = cv2.imread(pic_path2)
            ##############resize image
            width_re = 61
            height_re = 60
            img_small = cv2.resize(img,(width_re,height_re))
            img_small_grey = cv2.cvtColor(img_small,cv2.COLOR_BGR2GRAY)
            
            ##############get dHash array
            array = np.array(img_small_grey).flatten()
            h_array=[]
            for j in range(0,width_re * height_re):
                
                if(j%width_re == width_re-1):
                    continue
                else:
                    if(array[j] >= array[j+1]):
                        h_array.append(1)
                    else:
                        h_array.append(0)
            
            bin = '1111'
            for k in range(0,len(h_array)):
                
                bin = bin+str(h_array[k])                
            #print(format((hex(int(bin,2))),'x'))
            hex_hash = hex(int(bin,2))
            #print(len(hex_hash))
            
            
            pic_id.append(pic_id_value)
            v_hash.append(hex_hash)
        except:
            print('E2')
            pic_id.append(pic_id_value)
            v_hash.append(hex_hash)
        else:
            plt.show()
            
        
        #print(i)
    
    tmp['id'] = pic_id
    tmp['dHash_value'] = v_hash
    return tmp

In [50]:
def hex_to_binstring(hex_hash):
    
    a = bin(int(hex_hash,16))
    return a

In [67]:
def similarity(a,b):
    d = 0
    for i in range(0,3606):
        if(a[i]==b[i]):
            continue
        else:
            d = d+1
    return (1-d/3600)

In [68]:
product_list = image_list.product_id.unique()
product_sim_list = pd.DataFrame()
product_id_tmp = []
sim_tmp = []
count = 0

for j in product_list:
    #print(j)
    
    print(count)
    count = count+1
    
    image_list_j = image_list[image_list.product_id==j]
    seller_pic_num = image_list_j[image_list_j.from_who=='1'].shape[0] 
    #print(seller_pic_num)
    #for i in range(0, image_list_j.shape[0]):
    
    try:
        v_hash = []
        pic_id = []
        tmp = dHash(0,image_list_j.shape[0])
        image_list_j = pd.merge(image_list_j,tmp,on='id')
    except:
        continue
    #print(image_list_j)
    
    tmp=[]
    for p in range(0,seller_pic_num):
        try:
            s_bin = hex_to_binstring(image_list_j.at[p,'dHash_value'])
            #print(s_bin)
        except:
            tmp.append(0)
            break;
        #print(p)
        for l in range(seller_pic_num,image_list_j.shape[0]):
            try:
                b_bin = hex_to_binstring(image_list_j.at[l,'dHash_value'])
                sim = similarity(b_bin,s_bin)
                #print(sim)
                tmp.append(sim)
            except:
                tmp.append(0)
                continue
            
    mean_sim = (sum(tmp)/len(tmp)*1.0)
    sim_tmp.append(mean_sim)
    product_id_tmp.append(j)

    
product_sim_list['product_id']=product_id_tmp
product_sim_list['similarity_s_b'] = sim_tmp
display(product_sim_list)
product_sim_list.to_csv('/home/mk/Documents/Taobao/淘宝爬虫/毕业设计/product_pic_sim.csv')

0


,product_id,similarity_s_b
0,576039826635,0.511636


In [39]:
product_list = image_list.product_id.unique()
print(len(product_list))

98607
